In [1]:
## EDIT THIS CELL
x = 5      # Replace with the minimum number of times you want to sample your rarest cell type
p = 0.4    # Replace with the concentration of the rarest subtype in the form of a percent (0-100), ex. I would put 4 if the rarest cell type made up 4% of my sample
P = 99     # Replace this with the desired probability of sampling your rarest subtype x times in the form of a percent (0-100), ex. I would put 99 if I wanted a 99% probability of sampling the rarest subtype x times
d = 0      # Replace this with the expected sample dropout rate in the form of a percent (0-100), ex. I would put 15 if I expected 15% of my samples to end up not being viable enough for measurement

In [2]:
%load_ext rpy2.ipython
import math
import numpy as np
import textwrap

In [3]:
def find_n_guarantee(P = int, low = 0, high = 100000, x = int, p = int):
  # print(f"Low: {low}")
  # print(f"High: {high}")

  ## Base case
  if high >= low:
    n = (high + low) // 2
    # print(f"n: {n}")

    %Rpush x
    %Rpush n
    %Rpush p
    %R -o prob prob = 1 - pbinom(x, size = n, prob = p)

    result = prob[0]
    # print(f"Prob: {result}")

    if result == 1:
      return find_n(P, low, n - 1, x, p)
    else:
      return find_n(P, n + 1, high, x, p)

  else:
    return math.ceil((high + low) / 2)

In [4]:
def find_n(P = int, low = 0, high = 100000, x = int, p = int):
  # print(f"Low: {low}")
  # print(f"High: {high}")

  if P == 1:
    return find_n_guarantee(P = P, low = low, high = high, x = x, p = p)

  ## Base case
  if high >= low:
    n = (high + low) // 2
    # print(f"n: {n}")

    %Rpush x
    %Rpush n
    %Rpush p
    %R -o prob prob = 1 - pbinom(x, size = n, prob = p)

    result = prob[0]
    # print(f"Prob: {result}")

    if result == P:
      return n
    elif result > P:
      return find_n(P, low, n - 1, x, p)
    else:
      return find_n(P, n + 1, high, x, p)

  else:
    return math.ceil((high + low) / 2)

In [5]:
def adjust_n(n = int, d = float):
  return math.ceil(n / (1 - d))

In [7]:
n = find_n(P = P / 100, x = x, p = p / 100)  # Fix this to adjust high based on p
n_adj = adjust_n(n = n, d = d / 100)

print(textwrap.fill(f"In an 'infinite' population, the number of cells you need to randomly select and measure to have a {P}% probability of sampling each cell type at least {x:,} time(s) is {n_adj:,}, adjusting for sample dropout.", 60))

In an 'infinite' population, the number of cells you need to
randomly select and measure to have a 99% probability of
sampling each cell type at least 5 time(s) is 3,274,
adjusting for sample dropout.
